## Syncing your Fork

GitHub documentation: [Syncing a fork](https://docs.github.com/en/pull-requests/collaborating-with-pull-requests/working-with-forks/syncing-a-fork)

---

# Pandas Demonstration

In this demonstration we will look at (the first part) of a common pattern in bioinformatics:
filenames from experimental data (such as FASTA, or in this case LCMS) contain the 
experimental design information.

We shall look into workflows to process the larger data components in module 3 of this course.

In [1]:
import numpy as np
import pandas as pd

# from numpy.random import default_rng
# rng = default_rng()

import matplotlib.pyplot as plt
%matplotlib inline

Here I copy a tidy set of data from the pandas documentation.

In [15]:
import datetime

df = pd.DataFrame({
        "A": ["one", "one", "two", "three"] * 6,
        "B": ["A", "B", "C"] * 8,
        "C": ["foo", "foo", "foo", "bar", "bar", "bar"] * 4,
        "D": np.random.randn(24),
        "E": np.random.randn(24),
        "F": [datetime.datetime(2013, i, 1) for i in range(1, 13)] + [datetime.datetime(2013, i, 15) for i in range(1, 13)],
})

df

,A,B,C,D,E,F
0,one,A,foo,-0.181925,-2.192056,2013-01-01
1,one,B,foo,-0.350597,-0.303469,2013-02-01
2,two,C,foo,2.085241,0.144540,2013-03-01
3,three,A,bar,0.948744,0.536423,2013-04-01
4,one,B,bar,-0.466972,0.169964,2013-05-01
5,one,C,bar,-0.523165,-0.959000,2013-06-01
6,two,A,foo,-0.087809,0.712539,2013-07-01
7,three,B,foo,0.149242,0.158900,2013-08-01
8,one,C,foo,0.293783,-0.351797,2013-09-01
9,one,A,bar,0.357273,0.111074,2013-10-01


It is not uncommon to be provided an excel sheet that ends up looking something like the following
result of the call to `pd.pivot()`.

In [17]:
non_tidy_df = pd.pivot_table(df, index=["A", "B"], columns=["C"])
non_tidy_df

D                   E          
C             bar       foo       bar       foo
A     B                                        
one   A  0.220038  0.367078  0.592878 -2.290467
      B  0.037851  0.237920 -0.188141 -0.674667
      C -0.241570  0.779328 -0.621672 -0.031660
three A  0.146660       NaN  1.025941       NaN
      B       NaN -0.081549       NaN  0.514008
      C -0.285228       NaN -0.803010       NaN
two   A       NaN -0.243242       NaN  0.607244
      B -0.988485       NaN  0.232094       NaN
      C       NaN  0.919833       NaN  0.527684

The solution to this is to stack the data:

In [48]:
non_tidy_df.stack()

D         E
A     B C                      
one   A bar  0.220038  0.592878
        foo  0.367078 -2.290467
      B bar  0.037851 -0.188141
        foo  0.237920 -0.674667
      C bar -0.241570 -0.621672
        foo  0.779328 -0.031660
three A bar  0.146660  1.025941
      B foo -0.081549  0.514008
      C bar -0.285228 -0.803010
two   A foo -0.243242  0.607244
      B bar -0.988485  0.232094
      C foo  0.919833  0.527684

## Wrangling Data

Read our data file!

In [4]:
%%bash
ls data

filenames.txt


In [5]:
%%bash
# less data/filenames.txt

In pure Python we can read a file line by line, like this:

In [12]:
with open('data/filenames.txt', 'r') as file:
    print(file.readline())

02042021/Blank-r001.d



Convenience functions are provided in pandas for many common data types. Which type you
interact with most often will depend on the source of your data. 

I would hazard to guess that you will see text or .csv (comma separated value) files most often,
and that is what we will examine here.

I have copied the filenames from a current project and provided them, we shall try to prepare
the sample annotation data from these names and the provided specification.

### Provided Specification

```
[Grape Varitety]_[Smoke Event]_[Replicate]_[Treatment]
```

This means our output should be a data frame with (at least) those four columns.

In [13]:
pd.read_csv("data/filenames.txt")

,02042021/Blank-r001.d
0,02042021/Blank-r002.d
1,02042021/cONTROL 2.d
2,02042021/Control-r001.d
3,02042021/Control-r002.d
4,02042021/ME_S2C_R1_0.d
...,...
572,03232021/ME_S3C_R3_60.d
573,03232021/ME_S3C_R3_90.d
574,03232021/ME_S3C_R3_CO.d
575,03232021/Water blank-r001.d
